<a href="https://colab.research.google.com/github/SunnieCoder/Machine-Learning-projects/blob/main/face_recognition_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import math
!pip install pyheif
import pyheif

!pip install dlib
import dlib
!pip install face_recognition
!pip3 install scikit-learn
from sklearn import neighbors
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
import os

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'PNG', 'JPG', 'JPEG', 'HEIC', 'heic'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Trains the model: a k-nearest neighbors classifier for face recognition

def train(path, n_neighbors=None, knn_algo='ball_tree', verbose=True):

  # extract the student names and images from the import
  images = []
  classNames = []
  myList = os.listdir(path)

  for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    if curImg is None:
      print("Can't read", cl)

    face_bounding_boxes = face_recognition.face_locations(curImg)
    if len(face_bounding_boxes) != 1:
      # If there are no people (or too many people) in a training image, skip the image.
      if verbose:
        print("Image {} not suitable for training: {}".format(cl, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
  print("Student Name list:", classNames)
  print("Number of Students:", len(classNames))

  # encode all the training images
  encodeListKnown = []
  for img in images:
    try:
      if img is not None:
        face_bounding_boxes = face_recognition.face_locations(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img, known_face_locations=face_bounding_boxes)[0]
        encodeListKnown.append(encode)
      else:
        print(f"Failed to load image: {img}")
    except Exception as e:
       print(f"An error occurred while processing image: {img}")
       print(e)

  # Determine how many neighbors to use for weighting in the KNN classifier
  if n_neighbors is None:
      n_neighbors = int(round(math.sqrt(len(encodeListKnown))))
      if verbose:
          print("Chose n_neighbors automatically:", n_neighbors)

  # Create and train the KNN classifier
  knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
  knn_clf.fit(encodeListKnown, classNames)

  return knn_clf


In [ ]:
# Predict the validation images:
# Recognizes faces in given image using a trained KNN classifier

def predict(X_img, knn_clf=None, distance_threshold=0.6):

  X_face_locations = face_recognition.face_locations(X_img)

  # If no faces are found in the image, return an empty result.
  if len(X_face_locations) == 0:
    print("Didn't find a face in the image.")
    return []

  # Find encodings for faces in the test iamge
  faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

  # Use the KNN model to find the best matches for the test face
  closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)

  are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]
  print("Match with the object? ", are_matches)

  # Predict classes and remove classifications that aren't within the threshold
  return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]



In [ ]:
# main function
from PIL import Image, ImageDraw
if __name__ == "__main__":
    # STEP 1: Train the KNN classifier and save it to disk
    # Once the model is trained and saved, you can skip this step next time.

    # import the training images from the path
    path = 'drive/My Drive/SCU COEN 240 Machine Learning/imagelist'

    print("Training KNN classifier...")
    classifier = train(path, n_neighbors=2)
    print("Training complete!")
    print(" ")
    print("Let's start the magic!")

    # STEP 2: Using the trained classifier, make predictions for unknown images
    testPath = 'drive/My Drive/SCU COEN 240 Machine Learning/minTest'
    for image_file in os.listdir(testPath):
        full_file_path = os.path.join(testPath, image_file)

        if not os.path.isfile(full_file_path) or os.path.splitext(full_file_path)[1][1:] not in ALLOWED_EXTENSIONS:
          raise Exception("Invalid image path: {}".format(full_file_path))

        if image_file.split(".")[-1] in {'HEIC', 'heic'}:
          heif_file = pyheif.read(full_file_path)
          reImg = Image.frombytes(
            heif_file.mode,
            heif_file.size,
            heif_file.data,
            "raw",
            heif_file.mode,
            heif_file.stride,
          )
          X_img = np.array(reImg)

        else:
          X_img = face_recognition.load_image_file(full_file_path)

        print("---------------------------------------------")
        print("Looking for faces in [{}]".format(image_file))

        # Find all people in the image using a trained classifier model
        predictions = predict(X_img, knn_clf=classifier)
        print("Predict data: ", predictions)

        # Print results on the console
        for name, (top, right, bottom, left) in predictions:
            print("【RESULT】- Found {} at ({}, {})".format(name, left, top))
        print("---------------------------------------------")


Training KNN classifier...
Student Name list: ['Anirudh', 'Shreya Chinthala', 'Zhiyu', 'Zexin', 'Qihui', 'Samyuktha', 'Kaiyue', 'Yash', 'Prachi', 'Peiqi', 'Shreyas', 'Haochen', 'Manya', 'Griffin', 'Varshit', 'Haisong', 'Divyanth', 'Pooja', 'Kimsong', 'Husain', 'Wagawaththa', 'Shreya Devendra', 'Chen', 'Aparnaa', 'Wei', 'Ruthu', 'Yuhang', 'Xinze', 'Huiyu', 'Madhuri', 'Shubham', 'Yutong', 'Dheeraj']
Number of Students: 33
Training complete!
 
Let's start the magic!
---------------------------------------------
Looking for faces in [Screenshot 2023-10-21 at 9.59.56 PM.png]
Match with the object?  [True]
Predict data:  [('Anirudh', (55, 101, 107, 49))]
【RESULT】- Found Anirudh at (49, 55)
---------------------------------------------
---------------------------------------------
Looking for faces in [Screenshot 2023-10-06 at 12.28.55 PM.png]
Match with the object?  [True]
Predict data:  [('Shreyas', (32, 156, 94, 93))]
【RESULT】- Found Shreyas at (93, 32)
------------------------------------